# Sección B: Quién es Quién en los Precios

## Introducción

La siguiente asignación está referida al análisis de la base de datos de la Procuradoría Federal del Consumidor PROFECO, del programa Quien es Quién (QQP) que recaba y difunde información de precios de productos de consumo regular en el hogar, como alimentos, bebidas, productos de aseo personal y del hogar, medicinas, electrodomésticos y artículos de temporada para ofrecer información que permite tomar decisiones de compra mediante la comparación de precios. [1]

Esta base de datos es un registro histórico diario de más de 2,000 productos, a partir de 2015, en diversos establecimientos de la República Mexicana. Se pide realizar el análisis con PySpark SQL.

In [1]:
# Step 0. Cargar bibliotecas y funciones ·······················#
import pyspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext

In [2]:
# Step 1. Cargar los datos del archivo CSV ·····················#
# 1.1 Creamos primero un contexto
name = 'seccion b'
sc = SparkContext(appName = name)
sql_c = SQLContext(sc)
# 1.2 Y luego pasamos a leer el csv
df = sql_c.read.format('csv').options(header='true').load('all_data.csv')
sql_c.registerDataFrameAsTable(df, "my_table")
# 1.3 Es conveniente dar un vistazo a como está definido el esquema de la tabla que hemos generado
df.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [3]:
# Step 2. Creamos la vista temporal de la tabla ················#
df.createOrReplaceTempView("my_table")
# Damos una primera vista a los datos
sqlDF = sql_c.sql("SELECT * FROM my_table LIMIT 5")
sqlDF.show()
# Y pasamos a responder las preguntas

+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|   marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|      giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....|DISTRITO FEDERAL|TLALPAN          ...|19.29699|-99.125417|
|            CRAYONE

## 1. Procesamiento de los datos

### a. ¿Cuántos registros hay?

In [4]:
sqlDF = sql_c.sql("SELECT COUNT(*) FROM my_table")
sqlDF.show(2,False)

+--------+
|count(1)|
+--------+
|62530715|
+--------+



Respuesta: Hay 62,530,715 registros. Estos más de 62.5 millones de registros están dentro de la tabla creada con 15 variables.

### b. ¿Cuántas categorías?

In [5]:
sqlDF = sql_c.sql("SELECT COUNT(DISTINCT categoria) FROM my_table")
sqlDF.show(2,False)

+-------------------------+
|count(DISTINCT categoria)|
+-------------------------+
|41                       |
+-------------------------+



Respuesta: Hay 41 categorías en la variable "categoría".

### c.¿Cuántas cadenas comerciales están siendo monitoreadas?

In [6]:
sqlDF = sql_c.sql("SELECT COUNT(DISTINCT cadenaComercial) FROM my_table")
sqlDF.show(2,False)

+-------------------------------+
|count(DISTINCT cadenaComercial)|
+-------------------------------+
|705                            |
+-------------------------------+



Se están monitoreando 705 cadenas comerciales.

### d.¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de inconsistencia o error en la fuente?

Para iniciar, al haber hecho el vistazo del esquema de la tabla, pude observar que los precios están siendo tratados como una cadena (string), en vez de ser una variable numérica. De igual manera la fecha de registro, latitud y longitud se tratan como cadena. 

### e.¿Cuáles son los productos más monitoreados en cada entidad?

In [7]:
sqlDF = sql_c.sql("SELECT producto, COUNT(*) mycount FROM my_table GROUP BY producto ORDER BY mycount DESC LIMIT 10")
sqlDF.show()

+--------------------+-------+
|            producto|mycount|
+--------------------+-------+
|            REFRESCO|1247981|
|   DETERGENTE P/ROPA| 990122|
|                 FUD| 933410|
|LECHE ULTRAPASTEU...| 886716|
|             SHAMPOO| 745467|
|    JABON DE TOCADOR| 744914|
|      CHILES EN LATA| 724862|
|            MAYONESA| 697586|
|             YOGHURT| 632362|
|         DESODORANTE| 623684|
+--------------------+-------+



Podemos observar que los productos más monitoreados son los refrescos y detergentes. El caso de FUD puede referirse a una marca que encontré referida a carnes curadas y quesos en México. 

### f. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

In [8]:
sqlDF = sql_c.sql("SELECT cadenaComercial, COUNT(DISTINCT producto) AS counts \
FROM my_table GROUP BY cadenaComercial ORDER BY counts DESC LIMIT 10")
sqlDF.show()

+--------------------+------+
|     cadenaComercial|counts|
+--------------------+------+
|             SORIANA|  1059|
|            WAL-MART|  1051|
|MEGA COMERCIAL ME...|  1049|
|  COMERCIAL MEXICANA|  1036|
|            CHEDRAUI|  1026|
|     MERCADO SORIANA|  1024|
|      BODEGA AURRERA|  1012|
|HIPERMERCADO SORIANA|  1006|
|              H.E.B.|  1001|
|        SORIANA PLUS|   999|
+--------------------+------+



La cadena comercial con mayor variedad de productos monitoreados es Soriana.

## Referencias
 
[1] Gobierno de México - Profeco. (9 de mayo del 2021). *Quién es Quién en los Precios*. Descargado de https://www.profeco.gob.mx/precios/canasta/default.aspx  
